In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from collections import OrderedDict

In [3]:
train_df = pd.read_csv('dengue_features_train.csv')

In [4]:
label_df = pd.read_csv('dengue_labels_train.csv')

In [5]:
#merge label with feature df
train_data  = train_df.merge(label_df, 'left', on=['city', 'year', 'weekofyear'])

train_data.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [6]:
# remove feature week_start_date
train_data = train_data.drop('week_start_date', axis=1)
train_data.head()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [7]:
# binarize the city
one_hot = pd.get_dummies(train_data['city'])
# drop column city
train_data = train_data.drop('city', axis=1)
# add one hot encoded columns
train_data = train_data.join(one_hot)

train_data.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,iq,sj
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,...,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,0,1
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,...,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,1
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,...,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4,0,1
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,...,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3,0,1
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,...,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6,0,1


In [8]:
# check for NaN values
train_data.isna().sum()

year                                       0
weekofyear                                 0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                        20
station_min_temp_c                        14
station_precip_mm                         22
total_case

In [9]:
# impute all NaN with mean
train_data = train_data.fillna(method='backfill', axis=1)


In [10]:
train_data.isna().sum()

year                                     0
weekofyear                               0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
total_cases                              0
iq         

In [11]:
exclude_cols = ['year', 'weekofyear', 'total_cases', 'iq', 'sj']

In [12]:
cols = train_data.columns.tolist()

In [13]:
mean_dict = {}
sigma_dict = {}
for j in cols:
    if j not in exclude_cols:
        mu = train_data[j].mean()
        sigma = train_data[j].mean()
        mean_dict[j] = mu
        sigma_dict[j] = sigma
        train_data[j] = (train_data[j] - mu)/sigma

In [14]:
train_data['year'] = train_data['year']/2100
train_data['weekofyear'] = train_data['weekofyear']/100

In [15]:
# min max scaling the y variable
# y_min = train_data['total_cases'].min()
# y_max = train_data['total_cases'].max()

# train_data['total_cases'] = (train_data['total_cases'] - y_min)/(y_max - y_min)

In [16]:
# # using log scaling for y variable
# train_data['total_cases'] = np.log(train_data['total_cases'] + 1)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_tr, x_val, y_tr, y_val = train_test_split(train_data.drop('total_cases', axis=1).values,
                                           train_data['total_cases'].values,
                                           test_size=0.25,
                                           random_state=42)

In [19]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [24]:
model = xgb.XGBRegressor(objective ='gpu:reg:linear', 
                         colsample_bytree = 0.8, 
                         learning_rate = 0.01,
                         max_depth = 5, 
                         alpha = 1.5, 
                         n_estimators = 20)

In [25]:
model.fit(x_tr, y_tr)

XGBRegressor(alpha=1.5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=20,
       n_jobs=1, nthread=None, objective='gpu:reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [26]:
y_pred = model.predict(x_val)
y_pred = np.maximum(y_pred, 0)

In [22]:
# y_actual = (y_max - y_min)*y_val + y_min
# y_pred = (y_max - y_min)*y_pred + y_min

In [27]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 53.458059


In [28]:
y_out = pd.DataFrame()
y_out['Y'] = y_val
y_out['Yhat'] = y_pred

In [29]:
y_out.head(10)

,Y,Yhat
0,43.0,8.896951
1,6.0,1.755685
2,38.0,4.608071
3,7.0,1.426293
4,10.0,1.701687
5,22.0,2.720084
6,42.0,5.206533
7,9.0,1.876445
8,5.0,1.755685
9,8.0,2.007338


In [30]:
OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

OrderedDict([('f0', 84),
             ('f1', 50),
             ('f2', 31),
             ('f17', 21),
             ('f11', 19),
             ('f15', 17),
             ('f9', 13),
             ('f7', 12),
             ('f8', 10),
             ('f3', 10),
             ('f10', 7),
             ('f16', 7),
             ('f5', 6),
             ('f20', 6),
             ('f19', 5),
             ('f18', 4),
             ('f6', 3),
             ('f14', 2),
             ('f13', 2),
             ('f12', 2),
             ('f23', 1),
             ('f4', 1)])

In [31]:
#for tuning parameters
parameters_for_testing = {
   'colsample_bytree':[0.4,0.6,0.8],
   'gamma':[0,0.05,0.1,0.2],
   'min_child_weight':[5,6,7,8],
   'learning_rate':[0.01,0.05, 0.1],
   'max_depth':[4,5,6],
   'n_estimators':[100,150,200,250],
   'reg_alpha':[0.1, 0.15, 0.2],
   'reg_lambda':[0.1, 0.15, 0.2],
   'subsample':[0.6,0.8]  
}

In [35]:
xgb_model = xgboost.XGBRegressor(objective ='gpu:reg:linear',
                                 tree_method='gpu_hist', gpu_id=0,
                                 learning_rate =0.1, n_estimators=1000, max_depth=5,
                                 min_child_weight=1, gamma=0, subsample=0.8, 
                                 colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)

In [36]:
gsearch1 = GridSearchCV(estimator = xgb_model, 
                        param_grid = parameters_for_testing, 
                        n_jobs=-1,
                        iid=False, 
                        verbose=10,
                        scoring='neg_mean_squared_error')

In [37]:
gsearch1.fit(x_tr, y_tr)

Fitting 3 folds for each of 31104 candidates, totalling 93312 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   52.5s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  1

KeyboardInterrupt: 

In [ ]:
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)